In [1]:
!pip install -q streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 20.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 43.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 4.8 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
%%writefile app.py

import streamlit as st

import torch
import torch.nn as nn
import torch.nn.functional as F

import albumentations as A
from albumentations.pytorch import ToTensorV2

import io

import numpy as np
from PIL import Image

from torchvision.models import resnet34

st.set_page_config(layout="wide")


class ResNet34(nn.Module):
    def __init__(self, num_classes):
        super(ResNet34, self).__init__()
        self.model = resnet34(weights="DEFAULT")
        self.num_ftrs = self.model.fc.in_features
        self.model.fc = nn.Linear(self.num_ftrs, num_classes)
        self.model.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3)

    def forward(self, x):
        x = self.model(x)
        return x


def transform_image(image_bytes: bytes) -> torch.Tensor:
    image = Image.open(io.BytesIO(image_bytes))
    image = image.convert("L")
    image_array = np.array(image)

    test_transforms = A.Compose(
        [
            A.Resize(height=256, width=256),
            A.CLAHE(always_apply=False, p=1.0, clip_limit=(4, 4), tile_grid_size=(4, 4)),
            A.Normalize(mean=[0.5], std=[0.5], max_pixel_value=255.0, p=1.0),
            ToTensorV2(always_apply=True),
        ]
    )
    return test_transforms(image=image_array)["image"].unsqueeze(0)


def get_prediction(model, image_bytes):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    transformed_image = transform_image(image_bytes=image_bytes).to(device)
    outputs = model.forward(transformed_image)
    outputs = F.softmax(outputs, dim=1)
    return transformed_image, outputs


def main():
    st.title("Pneumonia Classification Solution")

    model = ResNet34(num_classes=2)

    model = torch.load("/content/drive/MyDrive/image_processing_project/model_cl.pt")
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    model.eval()

    uploaded_file = st.file_uploader(
        "Upload chest X-ray image", type=["jpg", "jpeg", "png"]
    )

    if uploaded_file:
        image_bytes = uploaded_file.getvalue()
        image = Image.open(io.BytesIO(image_bytes))

        st.image(image, caption="Uploaded Image")
        _, outputs = get_prediction(model, image_bytes)

        st.header(f"Possibility of pneumonia is {outputs[0][1]*100:0.2f}%")


if __name__ == "__main__":
    main()

Writing app.py


In [4]:
!ls

app.py	drive  sample_data


In [5]:
import urllib
print("Password/Enpoint IP for localtunnel is:",urllib.request.urlopen('https://ipv4.icanhazip.com').read().decode('utf8').strip("\n"))

Password/Enpoint IP for localtunnel is: 35.192.97.224


In [6]:
!npm install localtunnel

npm WARN saveError ENOENT: no such file or directory, open '/content/package.json'
npm notice created a lockfile as package-lock.json. You should commit this file.
npm WARN enoent ENOENT: no such file or directory, open '/content/package.json'
npm WARN content No description
npm WARN content No repository field.
npm WARN content No README data
npm WARN content No license field.

+ localtunnel@2.0.2
added 22 packages from 22 contributors and audited 22 packages in 1.706s

3 packages are looking for funding
  run `npm fund` for details

found 1 moderate severity vulnerability
  run `npm audit fix` to fix them, or `npm audit` for details


In [7]:
!streamlit run app.py &>/content/logs.txt &

In [ ]:
!npx localtunnel --port 8501

npx: installed 22 in 2.212s
your url is: https://spicy-moose-walk.loca.lt
